In [2]:
import os
import csv
import sys
import openpyxl
import time
# os.chdir(r'C:\Users\op1\Desktop\Pyfile\上手風控\data')
class productInfo:
    equityadm = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    marginadm = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    def __init__(self, contract=None, month=None,exchange=None,
                 Buy_oi=0,Sell_oi=0,tick=[0,0],margin=0,setP=0,currency=None,
                TDate=None,Broker='ADM'):
        
        self.contract = contract
        self.month = month
        self.exchange = exchange
        self.Buy_oi = Buy_oi
        self.Sell_oi = Sell_oi
        self.tick = tick
        self.margin = margin
        self.setP = setP
        self.currency = currency
        self.TDate = TDate
        self.Broker = Broker

    def change(self):
        with open('商品轉換.csv','rt',encoding = 'utf-8') as fphi:
            phifile = csv.reader(fphi)
            check_product = 0
            for i in phifile:
                if self.contract == i[0]:
                    self.contract = i[1]
                    self.exchange = i[2]
                    self.currency = i[3]
                    self.tick[0] = i[4]
                    self.tick[1]= i[5]
                    check_product += 1
                    break
            if check_product == 0 :
                print('請確認',self.contract,'找不到對照商品')
    def pimargin(self):
        with open('//10.36.205.170/交易結算部/國外/交易量+未平倉/' +date[5:]+'/保證金.TXT','r') as mar:
            marR = mar.readlines()
            for i in marR:
                if len(i.split())==11 and i.split()[2]==self.contract:
                    self.margin = int(i.split()[6].replace('.00','').replace(',',''))
                    


        
    
    def risk(contract,month,ticks=0):
        for i in admOI:
            if contract in i and month in i:
                profit = int(i[3])*int(i[4][0])*ticks/float(i[4][1])
        return(profit)
    
    def save(date):
        f1 = open(date.replace('/','-')+' admOI.csv','w+')
        for i in admOI:
            i.remove(i[4])
            f1.write(','.join(i)+'\n')
        f1.close()

    def piINFO(self):
        if self.exchange == 'LME':
            return([
                self.Broker ,
                self.contract,
                self.month,
                self.exchange, 
                self.Buy_oi,
                self.Sell_oi,
                #self.tick , 
                self.margin,
                abs((self.Buy_oi + self.Sell_oi) * self.margin),
                self.setP,
                self.currency,
                ])
        else:
            return([
                self.Broker ,
                self.contract,
                self.month,
                self.exchange, 
                self.Buy_oi,
                self.Sell_oi,
                #self.tick , 
                self.margin,
                abs((self.Buy_oi - self.Sell_oi) * self.margin),
                self.setP,
                self.currency])


#--------------------------------------------------上手資料

fileList = os.listdir()
fileList = (i for i in os.listdir() if i[-3:] == 'txt' and 'twnrpts' in i)

for i in fileList:
    adm= open(i, "r")
    admR = adm.readlines()
    adm.close()
    ct = 1
    ct2,ct3 = 1, 1
    date = ''
    sourceadm = []
    admOI=[]
    productInfo.equityadm = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    productInfo.marginadm = {'USD':0,'JPY':0,'SGD':0,'EUR':0,'HKD':0}
    
    exchange = ['CBOE','CBOT','CME','COMEX','NYBOT','NYMEX','SGX']
    # 找到 OPEN POSITION
    for l in range(len(admR)):
        if 'O  P  E  N ' in admR[l]:
            ct = l
            break

    for i in range(ct,len(admR)):
        if 'Business ' in admR[i]:
            date = admR[i].split()[3]
        if (len(admR[i].split())==11 or len(admR[i].split())==9 or \
        len(admR[i].split())==10 or  len(admR[i].split())==7 or  len(admR[i].split())==8 \
        or  len(admR[i].split())==12) and '--' not in admR[i].split()and 'ACCOUNT' not in admR[i].split():
            sourceadm.append(admR[i].split())   
    
    for i in range(ct,len(admR)):
        if 'MONEY' in admR[i]:        #權益數&保證金
            if ('F1' in admR[i] or 'F2' in admR[i] )and ct3<3:
                productInfo.equityadm['USD'] += int(float(admR[i].split()[-4].replace(',','')))
                ct3 += 1
            try:
                if 'N=' in admR[i]:
                    productInfo.equityadm['EUR'] = int(float(admR[i].split()[-4].replace(',','')))
                    productInfo.marginadm['EUR'] = int(float(admR[i+1].split()[-6].replace(',','')))
            except IndexError:
                productInfo.marginadm['EUR'] = int(float(admR[i+2].split()[-6].replace(',','')))
                
        if 'PBL' in admR[i] and ct2<3:
            productInfo.marginadm['USD'] += int(float(admR[i].split()[-6].replace(',','')))
            ct2 += 1
                   
           
    date0 = time.strptime(date, "%Y-%m-%d")
    date1 = time.strftime("%Y/%m/%d",date0)   #資料日期
    date = time.strftime("%Y-%m%d",date0)     #sheets名稱資料日期 




    for i in range(len(sourceadm)):
        if 'AV' in sourceadm[i]:
            pi = productInfo()
            pi.contract = sourceadm[i-1][6]
            pi.month = sourceadm[i-1][3]+sourceadm[i-1][4]
            if len(sourceadm[i-1]) ==9:
                pi.contract = sourceadm[i-1][5]
                pi.month = sourceadm[i-1][3]+sourceadm[i-1][4]
            elif len(sourceadm[i-1]) ==12:
                pi.month = sourceadm[i-1][4]+sourceadm[i-1][5]
            if sourceadm[i][1] != '*':
                pi.Sell_oi = int('-'+sourceadm[i][1].replace('*',''))
            if sourceadm[i][0] != '*':
                pi.Buy_oi = int(sourceadm[i][0].replace('*',''))
            if pi.contract == 'COCOA':
                pi.setP = str(int(float(sourceadm[i][4])*100))
            else:
                pi.setP = sourceadm[i][4]
            pi.change()
            pi.pimargin()
            admOI.append(pi.piINFO())
            
    curL = ['USD','JPY','EUR']
    for cur in curL:
        usdS = []
        usdN = []
        for i in admOI:
            if i[-1] ==cur:
                usdS.append(i[-3])
                usdN.append(i[1]+i[2])

        for i in range(len(admOI)):
            if cur in admOI[i]:
                marPPS = admOI[i][-3]/sum(usdS)
                admOI[i].append(marPPS)
                admOI[i].append(int(admOI[i][-1]*productInfo.marginadm[cur]))


               

    wb = openpyxl.load_workbook('RiskControl.xlsx')

    title=['上手','商品','月份','交易所','買方口數',
           '賣方口數','保證金','加總','結算價','幣別','占比','調整後保證金']

    ADMEqtAndMar = [
                    date1, pi.Broker,
                    productInfo.equityadm['USD'],productInfo.equityadm['EUR'],
                    productInfo.equityadm['JPY'],productInfo.equityadm['HKD'],
                    productInfo.equityadm['SGD'],
                    productInfo.marginadm['USD'],productInfo.marginadm['EUR'],
                    productInfo.marginadm['JPY'],productInfo.marginadm['HKD'],
                    productInfo.marginadm['SGD'],                
                   ]


    try:                      #判斷sheet是否存在
        ws = wb[date]
    except KeyError:
        wb.create_sheet(date)
        ws = wb[date]
        ws.append(title)

    t = 0                     #判斷資料是否存在
    for i in ws['A']:
        if 'ADM' in i.value:
            t=1
            break       
    if t == 0 :               #資料填入
        for row in range(len(admOI)):
            ws.append(admOI[row])

    ##....填回歷史資料

    ws = wb["HistoryData"]

    t = 0
    for i in ws['A']:         #判斷資料是否存在
        if date1 in i.value:
            if 'ADM' in ws.cell(row = i.row , column = 2).value:
                t=1
                break        
    if t == 0 :               #資料填入
        ws.append(ADMEqtAndMar)        

    wb.save('RiskControl.xlsx')
    print(pi.Broker+date+'OK')



ADM2021-0405OK
ADM2021-0406OK
ADM2021-0407OK
ADM2021-0419OK
ADM2021-0420OK
ADM2021-0421OK
ADM2021-0422OK
ADM2021-0423OK
